# Creating Bangalore Grid

Here, we will create a hexagonal grid of uniform size to cover the Bangalore city area. We will use data from the [Karanataka Government website](https://kgis.ksrsac.in/kgis/downloads.aspx)  (converted to GeoJSON [here](https://ogre.adc4gis.com/)), which has been retrieved and stored in the data folder.

In [1]:
import numpy as np
import pandas as pd
import geojson as gj
from geopy import Nominatim
import folium
import h3

In [2]:
# Creating an indicator to avoid repeating API calls when re-running parts of the notebook

# For a new session, just set this starting value to 1

# To skip API calls when re-running, set this to 0
# A cell can be added later in the notebook to set this variable to 1
# All flagged operations before that cell will be ignored, and all after will run

FLAG = 1

# Data Import

We first import a geoJSON file to describe the boundaries of the Bengaluru (Urban) area.

**Data Source:** OpenStreetMaps. We obtained the ID for Bengaluru (7902476) and then generated the polygon using this [tool](https://polygons.openstreetmap.fr/).

In [3]:
with open('../data/bangalore.geojson') as f:
    geodata = gj.load(f)

In [4]:
# Obtain centre coordinates for map
geocoder = Nominatim(user_agent = 'coursera_capstone')
if FLAG: map_centre = geocoder.geocode('Bengaluru, Karnataka, India')[1]

# Plot map
m = folium.Map(map_centre, zoom_start=11)
folium.Choropleth(geodata, name='geoJSON').add_to(m)

m # Display Map

## Creating Hexagonal grid

We wil use the [H3](https://github.com/uber/h3) geospatial system developed by Uber to create a grid of hex tiles covering the area described by our geoJSON file.

**Note:** For details of possible hex resolutions, see [link](https://h3geo.org/docs/core-library/restable).

In [6]:
# The current h3 python package function supports only a dict containing a Polygon
# Our file coontains a MultiPolygon with one element, so we just need to convert it to the required format

blr_poly = {'type': 'Polygon', 'coordinates': [list(gj.utils.coords(geodata))]}

In [7]:
hex_res = 8 # create hex grid of size 8 (Area=0.7373276 sq km, Edge=0.461354684 km)
hexagons = list(h3.polyfill(blr_poly, hex_res, geo_json_conformant=True)) 
print('{} Hexagon Grid generated.'.format(len(hexagons)))

1021 Hexagon Grid generated.


In [8]:
features_list = []
for hexagon in hexagons:
    hex_coords = list(h3.h3_to_geo_boundary(hexagon, geo_json = True))
    hex_coords.append(hex_coords[0]) # Add first point back to the end of the list to close the loop
    feature = gj.Feature(
        geometry = gj.Polygon([hex_coords]),
        id = hexagon,
        properties = {"resolution": hex_res},
    )
    features_list.append(feature)

blr_hexmap = gj.FeatureCollection(features_list)

## Plotting the grid

Check the grid before saving it.

In [9]:
# Plot map
m = folium.Map(map_centre, zoom_start=11)

# Draw Bangalore boundary area
folium.Choropleth(
    geodata,
    name='boundary',
    line_color = 'blue',
    line_opacity = 0.8,
    line_weight = 4,
    fill_color = 'blue',
    fill_opacity = 0.0,
).add_to(m)

# Draw hexes
folium.Choropleth(
    blr_hexmap,
    name='hex-grid',
    line_color = 'purple',
    line_opacity = 0.9,
    line_weight = 1,
    fill_color = 'purple',
    fill_opacity = 0.2,
).add_to(m)

m # Display map

We will save this grid to a geoJSON file to use later.

In [10]:
with open('../data/bangalore_hex_grid.geojson', 'w') as outfile:
    gj.dump(blr_hexmap, outfile)